In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetMobile,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetMobile(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1056)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1056)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   52.9s


[CV] ........................... C=0.01, score=0.867793, total=  51.5s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.843409, total=  56.6s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.859100, total=  58.0s
[CV] ........................... C=0.01, score=0.854331, total=  57.7s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.850980, total=  58.3s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.865480, total=  59.1s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.864173, total= 1.0min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.2min


[CV] ............................ C=0.1, score=0.869781, total= 4.2min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.860352, total= 4.3min
[CV] ............................ C=0.1, score=0.868186, total= 4.3min
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.881306, total= 4.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.860236, total= 4.5min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.865949, total= 4.5min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.875000, total= 4.6min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.853229, total= 4.8min
[CV] ................... C=1000000000.0, score=0.839567, total= 4.6min
[CV] ................... C=1000000000.0, score=0.853346, total= 4.6min
[CV] ................... C=1000000000.0, score=0.817013, total= 4.7min
[CV] ................... C=1000000000.0, score=0.857143, total= 4.6min
[CV] ................... C=1000000000.0, score=0.862240, total= 4.6min
[CV] ................... C=1000000000.0, score=0.849901, total= 4.5min
[CV] ................... C=1000000000.0, score=0.855578, total= 4.6min
[CV] .................. C=10000000000.0, score=0.833333, total= 4.2min
[CV] .................. C=10000000000.0, score=0.851272, total= 4.3min
[CV] .................. C=10000000000.0, score=0.836614, total= 4.1min
[CV] .................. C=10000000000.0, score=0.854331, total= 4.1min
[CV] .................. C=10000000000.0, score=0.866469, total= 4.2min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 24.4min remaining:   58.6s


[CV] .................. C=10000000000.0, score=0.832031, total= 4.7min
[CV] .................. C=10000000000.0, score=0.837463, total= 4.2min
[CV] .................. C=10000000000.0, score=0.857143, total= 4.2min
[CV] .................. C=10000000000.0, score=0.846918, total= 4.1min
[CV] .................. C=10000000000.0, score=0.844622, total= 4.5min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 25.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 56.25420089, 254.61145885, 288.9187669 , 289.27985413,
        291.9462894 , 287.88988338, 288.62203839, 288.77915456,
        285.90168839, 288.22679434, 286.06930351, 277.64095614,
        255.49377306]),
 'mean_score_time': array([0.05149856, 0.05072367, 0.0351721 , 0.0387115 , 0.03422513,
        0.04003847, 0.03626235, 0.02986825, 0.03379519, 0.03237453,
        0.03052323, 0.03161378, 0.0238605 ]),
 'mean_test_score': array([0.85635359, 0.86898185, 0.86237174, 0.85142068, 0.84717837,
        0.84905288, 0.85003946, 0.84707972, 0.84648777, 0.84757301,
        0.84964483, 0.84658642, 0.84599448]),
 'mean_train_score': array([0.89831677, 0.95344402, 0.98296473, 0.97817557, 0.97704672,
        0.9816383 , 0.98217423, 0.97852653, 0.9816833 , 0.98032572,
        0.97713008, 0.98151893, 0.97893009]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.85635, std: 0.00830, params: {'C': 0.01},
 mean: 0.86898, std: 0.00733, params: {'C': 0.1},
 mean: 0.86237, std: 0.00680, params: {'C': 1.0},
 mean: 0.85142, std: 0.00560, params: {'C': 10.0},
 mean: 0.84718, std: 0.01441, params: {'C': 100.0},
 mean: 0.84905, std: 0.01067, params: {'C': 1000.0},
 mean: 0.85004, std: 0.00612, params: {'C': 10000.0},
 mean: 0.84708, std: 0.00930, params: {'C': 100000.0},
 mean: 0.84649, std: 0.01190, params: {'C': 1000000.0},
 mean: 0.84757, std: 0.00949, params: {'C': 10000000.0},
 mean: 0.84964, std: 0.01347, params: {'C': 100000000.0},
 mean: 0.84659, std: 0.01305, params: {'C': 1000000000.0},
 mean: 0.84599, std: 0.01077, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.8690
